In [85]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import datetime


In [33]:
NUM_SENSORS = 4

In [66]:
def make_dataframes(n):

    df = pd.DataFrame()

    for i in range(n):

        x = pd.Series(ix for ix in range(10))

        sensor_df = pd.DataFrame()
        for j in range(NUM_SENSORS):
            sensor_df["sensor%d" % j] = np.sin(x*(i+(j*10)+1))

        df2 = pd.DataFrame({"ts": [datetime.datetime.now()], "sensor_data": [sensor_df]})
        df = pd.concat([df, df2], axis=0)
    return df

all_dfs = make_dataframes(3)
#df.shape[0]

In [91]:
widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Slider:',
    value=3
)

start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

slider = widgets.IntSlider(
    min=0,
    max=9,
    step=1,
    readout=False, 
)
btn_left = widgets.Button(icon="chevron-left")
btn_right = widgets.Button(icon="chevron-right")
control_box = widgets.Box([start_date_picker, btn_left, slider, btn_right])

fig = go.FigureWidget()

df = all_dfs.iloc[0]
x=list(range(0,df['sensor_data'].shape[0]))
for i in range(NUM_SENSORS):
    fig.add_trace(go.Scatter(x=[], y=[], name="sensor%d" % i))


def draw(idx):
    print("draw", idx)
    df = all_dfs.iloc[idx]
    fig.update_layout({"title": f"Dataset from {df['ts']}"})

    for i in range(NUM_SENSORS):
        fig.data[i].x = list(range(0,df['sensor_data'].shape[0]))
        fig.data[i].y = df["sensor_data"]["sensor%d" % i]



def handle_slider_change(change):
    draw(change.new)

def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))

slider.observe(handle_slider_change, names='value')
btn_right.on_click(btn_eventhandler)

vbox = widgets.VBox([control_box, fig])
display(vbox)
draw(0)


draw 0


In [70]:
df = all_dfs.iloc[0]

a = list(range(0,df['sensor_data'].shape[0]))
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]